# Estimating genome-wide haplotype frequencies using LightGBM 

### Overall workflow:

1) Generate simulated populations with recombination/drift with simulate_population. Takes as input a matrix of genotypes at fixed intervals for population founders (each column is a population).
2) Calculate true haplotype frequencies in simulated populations for model training with get_true_freqs
3) Generate simulated NGS data from populations with generate_reads
4) Map reads, call SNPs in populations and founder lines
5) Define genomic windows and identify true haplotype frequency per window with either define_windows or define_by_SNPs
6) Train model using observed SNP frequencies and true haplotype frequencies per window 
7) Evaluate model performance, visualize predicted/true frequencies


In [ ]:
#have to run python 3.10 for now, biopython does not yet support 3.13

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pysam
from Bio import SeqIO
import random
from itertools import combinations
import matplotlib.pyplot as plt
import lightgbm as lgb


## Load RIL genotype data and pivot to wide format

In [ ]:
data = pd.read_csv('/Users/tyler/Desktop/haplotype_ML/chr3L_RILs_updated.csv')
df = pd.DataFrame(data)
#set RILs to df columns
df_wide = df.pivot_table(index=['CHROM', 'pos'], columns='sample', values='fHap', aggfunc='first')
df_wide.head()

## Replacing the old `recombine`, `simulate_population`, and `get_true_freqs` with updated, optimized classes

Below is the new **Chromosome**, **Population**, **Simulation** structure, which handles recombination, population simulation, and true haplotype frequency calculation, preserving the logic of the old code.

In [ ]:
# recombination function to be used in simulate_population
# function to simulate population w/drift
# determines true haplotype frequency in simulated population

import random

class Chromosome:
    def __init__(self, genotype):
        self.genotype = genotype

    @staticmethod
    def recombine(chrom1, chrom2):
        # randomly select a recombination point (window) between the chromosomes at 10,000bp interval
        recombination_point = random.randrange(0, len(chrom1.genotype) - 1)
        
        # create the offspring chromosome with recombination
        new_chrom1 = pd.concat([
            chrom1.genotype.iloc[:recombination_point],
            chrom2.genotype.iloc[recombination_point:]
        ])
        new_chrom2 = pd.concat([
            chrom2.genotype.iloc[:recombination_point],
            chrom1.genotype.iloc[recombination_point:]
        ])
        
        return Chromosome(new_chrom1), Chromosome(new_chrom2)

class Population:
    def __init__(self, ril_matrix, population_size):
        self.ril_matrix = ril_matrix
        self.population_size = population_size
        # Initiate starting population
        self.chromosomes = [
            Chromosome(ril_matrix.iloc[:, idx])
            for idx in np.random.choice(ril_matrix.shape[1], population_size * 2, replace=True)
        ]

    def simulate_generation(self, recombination_rate):
        new_chromosomes = []
        for _ in range(self.population_size):
            parent1, parent2 = random.sample(self.chromosomes, 2)
            
            if random.random() < recombination_rate:
                offspring1, offspring2 = Chromosome.recombine(parent1, parent2)
            else:
                # no recombination, copy parents
                offspring1, offspring2 = Chromosome(parent1.genotype.copy()), Chromosome(parent2.genotype.copy())
            
            new_chromosomes.extend([offspring1, offspring2])

        # drift by random sampling
        self.chromosomes = random.choices(new_chromosomes, k=self.population_size * 2)

    def simulate_generations(self, n_generations, recombination_rate):
        for generation in range(n_generations):
            self.simulate_generation(recombination_rate)

    def get_population_matrix(self):
        return pd.concat([chrom.genotype for chrom in self.chromosomes], axis=1)

class Simulation:
    def __init__(self, ril_matrix, n_flies, n_generations, recombination_rate):
        self.population = Population(ril_matrix, n_flies)
        self.n_generations = n_generations
        self.recombination_rate = recombination_rate

    def run(self):
        # simulate population with drift/recombination
        self.population.simulate_generations(self.n_generations, self.recombination_rate)
        return self.population.get_population_matrix()

    def calculate_haplotype_frequencies(self):
        # calculates haplotype frequencies in simulated population
        simulated_pop = self.population.get_population_matrix()
        haplotype_columns = simulated_pop.columns.difference(['sample', 'CHROM', 'pos'])
        haplotype_counts = simulated_pop[haplotype_columns].apply(lambda x: x.value_counts(), axis=1).fillna(0)
        haplotype_frequencies = haplotype_counts.div(haplotype_counts.sum(axis=1), axis=0)
        return haplotype_frequencies


## Replacing the old read_fasta, read_coordinates, reverse_complement, generate_reads with the new `GenomeSimulator`

In [ ]:
#should be one fasta file per chromosome, each containing  all the full set of template sequences (i.e. founder chromosomes)
#coordinates to simulated PE reads, adjust average read length as desired
#generate reverse complement for PE2
#outputs PE reads

class GenomeSimulator:
    def __init__(self, fasta_files, chromosomes):
        # store founder fastas for each chromosome
        self.chromosome_dict = {
            chrom: self._read_fasta(fasta_files[chrom]) for chrom in chromosomes
        }
        self.chromosomes = chromosomes

    @staticmethod
    def _read_fasta(file_path):
        sequences = {}
        for record in SeqIO.parse(file_path, "fasta"):
            haplotype_id = record.id
            sequences[haplotype_id] = record.seq
        return sequences

    def _reverse_complement(self, seq):
        complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
        return ''.join(complement.get(base, base) for base in reversed(seq))

    def _read_coordinates(self, population, chroms):
        # get read length from normal distribution
        read_length_f = np.round((np.random.normal(loc=150, scale=10))).astype(int)
        read_length_r = np.round((np.random.normal(loc=150, scale=10))).astype(int)

        # library fragment length and inner distance
        fragment = np.round((np.random.normal(loc=500, scale=50))).astype(int)
        gap = fragment - (read_length_f + read_length_r)

        # select chromosome to read
        chromosome = np.random.choice(chroms, replace=True)
        chrom_of_interest = population.loc[chromosome]
        max_pos = chrom_of_interest.index.max()

        # define read boundaries
        read_start_f = np.random.randint(1, max_pos - 2000)
        read_end_f = read_start_f + read_length_f
        read_end_r = read_end_f + gap
        read_start_r = read_end_r + read_length_r

        return chromosome, read_start_f, read_end_f, read_length_f, read_start_r, read_end_r, read_length_r

    def generate_paired_end_reads(self, population, read_num, out_name):
        # store number of reads generated per haplotype per position (read depth)
        haplotype_counts = pd.DataFrame(0, index=population.index, columns=['B1','B2','B3','B4','B5','B6','B7','B8'])

        read_count = 0
        chrom_list = list(self.chromosome_dict.keys())

        # initialize empty fastq files
        with open(f'{out_name}_1.fastq', 'w') as fastq_file1, open(f'{out_name}_2.fastq', 'w') as fastq_file2:

            while read_count < read_num:
                # randomly define read coordinates
                chrom, start_f, end_f, length_f, start_r, end_r, length_r = self._read_coordinates(population, chrom_list)

                # subset population dataframe to chromosome where read is
                chrom_of_interest = population.loc[chrom]

                # pick random chromosome to sequence
                template = chrom_of_interest.iloc[:, np.random.randint(1, chrom_of_interest.shape[1])]

                # get haplotype at position nearest to read location
                nearest_pos = min(template.index, key=lambda x: abs(x - start_f))
                haplotype = template[nearest_pos]

                # generate read
                fasta_seqs = self.chromosome_dict[chrom]
                haplotype_sequence = fasta_seqs[haplotype]

                forward_read = haplotype_sequence[start_f:end_f]
                reverse_read = haplotype_sequence[end_r:start_r]
                reverse_read = self._reverse_complement(reverse_read)
                reverse_read = reverse_read[::-1]  # flip reverse the sequence

                # track reads per haplotype
                haplotype_counts.at[(chrom, nearest_pos), haplotype] += 1
                read_count += 1
                read_ID = np.random.randint(1000, 9999)

                # write read to fastq file:
                read_quality_f = 'I' * len(forward_read)
                fastq_file1.write(f'@{chrom}_{start_f}_{read_ID}/1\n')
                fastq_file1.write(f'{forward_read}\n')
                fastq_file1.write('+\n')
                fastq_file1.write(f'{read_quality_f}\n')

                read_quality_r = 'I' * len(reverse_read)
                fastq_file2.write(f'@{chrom}_{start_f}_{read_ID}/2\n')
                fastq_file2.write(f'{reverse_read}\n')
                fastq_file2.write('+\n')
                fastq_file2.write(f'{read_quality_r}\n')

        return haplotype_counts, fastq_file1, fastq_file2


## Defining genomic windows with either window size or number of SNPs
Below we replace `define_windows` and `define_by_SNPs` with a refactored `GenomicWindow` class.

In [ ]:
#returns dictionary where key = genomic position of window start/end and values: 
#window: 9 x N matrix (N = number of snps) and true_freqs (true haplotype frequencies of that window)
#just like the old define_windows and define_by_SNPs, but neatly wrapped in a class with 2 methods

class GenomicWindow:
    def __init__(self, observed_frequencies, true_frequencies):
        self.observed_frequencies = observed_frequencies
        self.true_frequencies = true_frequencies

    def define_windows(self, window, step, sim):
        # define windows by physical size
        results = {}
        for window_start in range(self.observed_frequencies['pos'].min(), 
                                  self.observed_frequencies['pos'].max() - window, step):
            window_end = window_start + window
            window_data = self.observed_frequencies[
                (self.observed_frequencies['pos'] >= window_start) & 
                (self.observed_frequencies['pos'] < window_end)
            ]
            #minimum number of SNPs per window
            if window_data.shape[0] > 20:
                middle_obs = window_data['pos'].median()
                window_data_trimmed = window_data.iloc[:, 2:11].reset_index(drop=True)
                middle_true = min(self.true_frequencies['pos'], key=lambda x: abs(x - middle_obs))
                window_true_freq = self.true_frequencies[self.true_frequencies['pos'] == middle_true].iloc[:, 2:10].reset_index(drop=True)
                results[str(sim), (str(window_start), str(window_end))] = {
                    'window': window_data_trimmed,
                    'true_freq_row': window_true_freq
                }
        return results

    def define_by_SNPs(self, snp_number, step, sim):
        # define windows by SNP count
        results = {}
        window_start = 0
        window_end = window_start + snp_number

        while window_end < self.observed_frequencies.index.max():
            window_data = self.observed_frequencies.iloc[window_start:window_end, :]
            middle_obs = window_data['pos'].median()
            window_data_trimmed = window_data.iloc[:, 2:11].reset_index(drop=True)
            middle_true = min(self.true_frequencies['pos'], key=lambda x: abs(x - middle_obs))
            window_true_freq = self.true_frequencies[self.true_frequencies['pos'] == middle_true].iloc[:, 2:10].reset_index(drop=True)
            results[
                str(sim), (
                    str(self.observed_frequencies.iloc[window_start]['pos']), 
                    str(self.observed_frequencies.iloc[window_end]['pos'])
                )
            ] = {
                'window': window_data_trimmed,
                'true_freq_row': window_true_freq
            }
            window_start += step
            window_end += step
        return results


## Setting simulation parameters and generating reads

In [ ]:
n_flies = 300  # population size
n_generations = 15  # number of generations
recombination_rate = 0.5  # probability of recombination occurring
read_num = 500000 # when generating data for chr3L only this simulates ~100X read depth
num_sims = 100 # number of populations to create

# list of chromosomes to sample (including only 3L for now, will add the other pending genotyping data)
chroms = ['chr3L']

# dict so that the correct haplotype can be called from a randomly selected chromosome by read_coordinates
# (Now embedded in GenomeSimulator)
fasta_files = {
    'chr3L': '/Users/tyler/Desktop/haplotype_ml/founder_fastas/B.3L.fasta'
}

### Running the simulation for multiple populations, generating reads

In [ ]:
genome_simulator = GenomeSimulator(fasta_files, chroms)

for i in range(1, num_sims + 1):
    sim_name = f'sim{i}'
    
    # simulate population
    sim = Simulation(df_wide, n_flies, n_generations, recombination_rate)
    sim_pop = sim.run()
    true_freqs = sim.calculate_haplotype_frequencies()

    # generate reads
    read_depth, fq1, fq2 = genome_simulator.generate_paired_end_reads(sim_pop, read_num, sim_name)
    read_depth.to_csv(f'{sim_name}_depth.csv', index=True)
    true_freqs.to_csv(f'{sim_name}_true_freqs.csv', index=True)


### Example of calling SNP frequencies and defining genomic windows

In [ ]:
# sample code to parse SNP calls (SNP_freqs), define windows, etc.
# in practice you'd do bcftools or similar to generate a .csv of SNP frequencies

col_names = ['chrom', 'pos', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']
for i in range(1, num_sims + 1):
    sim_name = f'sim{i}'
    col_names.append(sim_name)

# hypothetical example:
# SNP_freqs = pd.read_csv('sim100_freqs.csv', names=col_names, header=None)
# input_windows = {}
# for i in range(1, num_sims+1):
#     sname = f'sim{i}'
#     true_freqs = pd.read_csv(f'{sname}_true_freqs.csv')
#     obs_freqs = SNP_freqs[['chrom','pos','B1','B2','B3','B4','B5','B6','B7','B8', sname]].copy()
#     obs_freqs['pos'] = obs_freqs['pos'].round(-3)
#     gw = GenomicWindow(obs_freqs, true_freqs)
#     windows = gw.define_windows(200000, 20000, sname)
#     input_windows.update(windows)
# len(input_windows)


### Model training example using LightGBM (unchanged)
Below is your original model training pipeline, referencing `input_windows` as prepared above.

In [ ]:
# The rest of your original ML training code remains unchanged.
# For instance:

# X_data_agg = []
# y_data = []
# for key, value in input_windows.items():
#     # etc.
#     pass

print("Notebook has replaced old scripts with updated classes. Ready for end-to-end workflow.")